In [1]:
import pandas as pd
import matplotlib.pyplot as ply
import seaborn as sns
import numpy as np
import scipy.stats as st
import statsmodels.formula.api as smf

In [2]:
import pandas as pd
import matplotlib.pyplot as ply
import seaborn as sns
import numpy as np
import scipy.stats as st
# Load libraries
import pandas as pd
#import numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
import statsmodels.formula.api as smf
from sklearn import linear_model
from patsy.builtins import Q
from sklearn.linear_model import RidgeCV

In [3]:
df = pd.read_csv("LoanStats3b.csv",low_memory =False)

In [4]:
#df.head(10)


In [5]:
print('Number of missing values per column:')
countMissing = df.isnull().sum()
print(countMissing)

Number of missing values per column:
id                                            188181
member_id                                     188181
loan_amnt                                          0
funded_amnt                                        0
funded_amnt_inv                                    0
term                                               0
int_rate                                           0
installment                                        0
grade                                              0
sub_grade                                          0
emp_title                                      11737
emp_length                                      7887
home_ownership                                     0
annual_inc                                         0
verification_status                                0
issue_d                                            0
loan_status                                        0
pymnt_plan                                         0
url      

In [6]:
# Drop the columns where all elements are missing
#df = df.dropna(axis=0,how='any')
df = df.dropna(axis=1,how='all')


In [7]:
# Columns that are being dropped
df=df.drop(['hardship_dpd','hardship_loan_status','emp_title','hardship_type','hardship_reason','hardship_status','deferral_term','hardship_amount'],axis=1)
df=df.drop(['settlement_status','settlement_date','settlement_amount','settlement_term', 'policy_code', 'acc_now_delinq', 'num_tl_30dpd'],axis=1)
df=df.drop(['hardship_start_date','hardship_end_date','orig_projected_additional_accrued_interest','hardship_payoff_balance_amount','debt_settlement_flag_date'],axis=1)
df=df.drop(['total_bal_ex_mort', 'num_sats', 'tot_cur_bal', 'tax_liens', 'zip_code','addr_state','title', 'num_tl_90g_dpd_24m'], axis=1)
df=df.drop(['payment_plan_start_date','hardship_length','hardship_last_payment_amount','settlement_percentage', 'collections_12_mths_ex_med'],axis=1)
df=df.drop(['mths_since_last_record', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'mths_since_last_delinq' ,'mths_since_last_major_derog'],axis=1)
df=df.drop(['hardship_flag', 'debt_settlement_flag', 'num_tl_120dpd_2m', 'chargeoff_within_12_mths', 'delinq_amnt', 'application_type', 'emp_length'],axis=1)
df=df.drop(['out_prncp', 'out_prncp_inv'], axis=1)

In [8]:
print('Number of missing values per column:')
countMissing = df.isnull().sum()
print(countMissing)

Number of missing values per column:
loan_amnt                          0
funded_amnt                        0
funded_amnt_inv                    0
term                               0
int_rate                           0
installment                        0
grade                              0
sub_grade                          0
home_ownership                     0
annual_inc                         0
verification_status                0
issue_d                            0
loan_status                        0
pymnt_plan                         0
desc                          106703
purpose                            0
dti                                0
delinq_2yrs                        0
earliest_cr_line                   0
inq_last_6mths                     0
open_acc                           0
pub_rec                            0
revol_bal                          0
revol_util                       125
total_acc                          0
initial_list_status                0
t

In [9]:
# df = df.drop(df[df['revol_util']==0].index)
# df.drop(df.index[df['last_credit_pull_d'] == 0], inplace = True)

In [9]:
df['int_rate'] = df['int_rate'].str.rstrip('%')
df['int_rate'] = df['int_rate'].astype('float64')

In [10]:
numericalList = []
nonNumList = []
for column in df.columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
        numericalList.append(column)
    else:
        nonNumList.append(column)

In [11]:
numDF = df[numericalList]
nonnumDF = df[nonNumList]

In [12]:
nrow = len(numDF['int_rate'])
# print('Number of missing values per column:')
num_countMissing = numDF.isnull().sum()
# print(num_countMissing)

In [13]:
# Estimate the mean for columns that are missing less than 20% of the observations
for coln in numDF.columns:
    if num_countMissing[coln] != 0:
        if num_countMissing[coln]/nrow < 0.20:
            numDF[coln] = numDF[coln].fillna(value=numDF[coln].mean())

/Users/michellebaginski/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [14]:
# remove all the observations with NAs in them
df = df.dropna(axis=0,how='any')

print('Number of missing values per column:')
countMissing = df.isnull().sum()
print(countMissing)

Number of missing values per column:
loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
term                          0
int_rate                      0
installment                   0
grade                         0
sub_grade                     0
home_ownership                0
annual_inc                    0
verification_status           0
issue_d                       0
loan_status                   0
pymnt_plan                    0
desc                          0
purpose                       0
dti                           0
delinq_2yrs                   0
earliest_cr_line              0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
revol_util                    0
total_acc                     0
initial_list_status           0
total_pymnt                   0
total_pymnt_inv               0
total_rec_prncp               0
total_rec_int                 0
   

In [15]:
# Removing variables on the basis of correlation matrix
corr = numDF.corr()
print(corr)
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.8:
            if columns[j]:
                columns[j] = False
selected_columns = numDF.columns[columns]


                            loan_amnt  funded_amnt  funded_amnt_inv  int_rate  \
loan_amnt                    1.000000     0.999799         0.999663  0.182654   
funded_amnt                  0.999799     1.000000         0.999874  0.182485   
funded_amnt_inv              0.999663     0.999874         1.000000  0.182933   
int_rate                     0.182654     0.182485         0.182933  1.000000   
installment                  0.955011     0.955254         0.955211  0.165173   
annual_inc                   0.368164     0.368151         0.368084 -0.026026   
dti                          0.044557     0.044572         0.044746  0.147471   
delinq_2yrs                  0.011184     0.011214         0.011391  0.097230   
inq_last_6mths               0.019741     0.019703         0.020091  0.241345   
open_acc                     0.191571     0.191614         0.191719  0.017359   
pub_rec                     -0.073588    -0.073537        -0.073373  0.056575   
revol_bal                   

In [17]:
numDF = numDF[selected_columns]



In [18]:
# £print(nonnumDF[['grade','home_ownership','loan_status','purpose','sub_grade','term']])

In [19]:
# Quartiles and IQR for mths_since_recent_bc

for col in numDF.columns:
    quartiles = numDF[col].quantile([0.25, 0.75], interpolation='nearest')
    q1 = quartiles[0.25]
    q3 = quartiles[0.75]
    IQR = q3 - q1
    outlier_val = q3 + 1.5*IQR
    #print("Outlier val:", outlier_val)
    numDF[col] = np.where(numDF[col] > outlier_val, outlier_val, numDF[col])

#df1 = df['mths_since_recent_bc'] = np.where(df['mths_since_recent_bc'] > outlier_val, outlier_val, df['mths_since_recent_bc'])

In [17]:
finaldf = pd.concat([numDF, nonnumDF[['grade','home_ownership','loan_status','purpose','sub_grade','term']]], axis=1)

In [18]:
print('Number of missing values per column:')
countMissing = finaldf['loan_status']
count = 0
ncount = 0
for w in countMissing:
    if w == "Fully Paid":
        count+=1
    else:
        ncount+=1
print(count/len(finaldf['loan_status']))
print(ncount/len(finaldf['loan_status']))

Number of missing values per column:
0.8423273337903402
0.15767266620965986


In [19]:
#making categorical variable dummy variable 
finaldf = pd.get_dummies(finaldf, columns=['grade','home_ownership','loan_status','purpose','sub_grade','term'])

In [20]:
print(finaldf.columns[100:])

Index(['sub_grade_E4', 'sub_grade_E5', 'sub_grade_F1', 'sub_grade_F2',
       'sub_grade_F3', 'sub_grade_F4', 'sub_grade_F5', 'sub_grade_G1',
       'sub_grade_G2', 'sub_grade_G3', 'sub_grade_G4', 'sub_grade_G5',
       'term_ 36 months', 'term_ 60 months'],
      dtype='object')


In [21]:
## Dividing dataset into 2 parts
x_train, x_test, y_train, y_test = train_test_split(finaldf.loc[:, finaldf.columns != 'int_rate'], finaldf['int_rate'], test_size=0.3, random_state = 0)

In [22]:
scaler = StandardScaler()
scaler.fit(x_train)
scaler.fit(x_test)
names = x_train.columns
x_train_scaled = scaler.transform(x_train)
x_train_scaled = pd.DataFrame(x_train_scaled, columns=names)
x_test_scaled = scaler.transform(x_test)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=names)

In [23]:
# Build model with only numeric variables (using statsmodels)
#df=df.rename(columns = {"Percent Less than Bachelors Degree":Percent Less than Bachelor Degree})
model = smf.ols(formula = 'int_rate ~ loan_amnt + annual_inc + dti + delinq_2yrs + inq_last_6mths+open_acc + pub_rec + revol_bal + total_acc + total_rec_late_fee+ recoveries + last_pymnt_amnt + tot_coll_amt + acc_open_past_24mths + avg_cur_bal + bc_open_to_buy +bc_util + mo_sin_old_il_acct + mo_sin_old_rev_tl_op + mo_sin_rcnt_rev_tl_op + mo_sin_rcnt_tl + mort_acc + mths_since_recent_bc + mths_since_recent_inq + num_accts_ever_120_pd + num_actv_bc_tl + num_actv_rev_tl + num_bc_tl + num_il_tl + num_tl_op_past_12m + pct_tl_nvr_dlq  + total_il_high_credit_limit', data = finaldf)
results = model.fit() 
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               int_rate   R-squared:                       0.411
Model:                            OLS   Adj. R-squared:                  0.411
Method:                 Least Squares   F-statistic:                     4103.
Date:                Mon, 02 Dec 2019   Prob (F-statistic):               0.00
Time:                        15:50:07   Log-Likelihood:            -4.9761e+05
No. Observations:              188181   AIC:                         9.953e+05
Df Residuals:                  188148   BIC:                         9.956e+05
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [24]:
# Build model with full dataset and removing varibales on the ba (using statsmodels)
#df=df.rename(columns = {"Percent Less than Bachelors Degree":Percent Less than Bachelor Degree})
model = smf.ols(formula = 'int_rate ~ loan_amnt + annual_inc + dti + delinq_2yrs + inq_last_6mths+open_acc + pub_rec + revol_bal + total_acc + total_rec_late_fee+ recoveries + last_pymnt_amnt + tot_coll_amt + acc_open_past_24mths + avg_cur_bal + bc_open_to_buy +bc_util + mo_sin_old_il_acct + mo_sin_old_rev_tl_op + mo_sin_rcnt_rev_tl_op + mo_sin_rcnt_tl + mort_acc + mths_since_recent_bc + mths_since_recent_inq + num_accts_ever_120_pd + num_actv_bc_tl + num_actv_rev_tl + num_bc_tl + num_il_tl + num_tl_op_past_12m + pct_tl_nvr_dlq  + total_il_high_credit_limit + grade_A+ grade_B+ grade_C+ grade_D+ grade_E+ grade_F+ grade_G+ home_ownership_MORTGAGE \
                + home_ownership_NONE+ home_ownership_OTHER+ home_ownership_OWN+ \
       home_ownership_RENT+ Q("loan_status_Charged Off") \
       + Q("loan_status_Fully Paid") + purpose_car+ purpose_credit_card \
        + purpose_debt_consolidation+ purpose_home_improvement \
        + purpose_house+ purpose_major_purchase+ purpose_medical \
       + purpose_moving+ purpose_other+ purpose_renewable_energy \
        + purpose_small_business+ purpose_vacation+ purpose_wedding \
       + sub_grade_A1+ sub_grade_A2+ sub_grade_A3+ sub_grade_A4 \
       + sub_grade_A5+ sub_grade_B1+ sub_grade_B2+ sub_grade_B3 \
       + sub_grade_B4+ sub_grade_B5+ sub_grade_C1+ sub_grade_C2 \
       + sub_grade_C3+ sub_grade_C4+ sub_grade_C5+ sub_grade_D1 \
       + sub_grade_D2+ sub_grade_D3+ sub_grade_D4+ sub_grade_D5 \
       + sub_grade_E1 + sub_grade_E2 + sub_grade_E3 + sub_grade_E4+ sub_grade_E5+ sub_grade_F1+ sub_grade_F2\
       + sub_grade_F3+ sub_grade_F4+ sub_grade_F5+ sub_grade_G1\
       + sub_grade_G2+ sub_grade_G3+ sub_grade_G4+ sub_grade_G5', data = finaldf)
results = model.fit() 
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               int_rate   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 3.634e+05
Date:                Mon, 02 Dec 2019   Prob (F-statistic):               0.00
Time:                        15:50:55   Log-Likelihood:                -69087.
No. Observations:              188181   AIC:                         1.383e+05
Df Residuals:                  188097   BIC:                         1.392e+05
Df Model:                          83                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

In [25]:
# Build model after removing variables on the basis of p value(using statsmodels)
#df=df.rename(columns = {"Percent Less than Bachelors Degree":Percent Less than Bachelor Degree})
model = smf.ols(formula = 'int_rate ~ loan_amnt  + dti  + inq_last_6mths + pub_rec +bc_util + num_bc_tl + num_tl_op_past_12m + grade_A+ grade_B+ grade_C+ grade_D+ grade_E+ home_ownership_MORTGAGE \
                + home_ownership_OTHER+ home_ownership_OWN+ \
       home_ownership_RENT+ Q("loan_status_Charged Off") \
        + purpose_credit_card \
        + purpose_debt_consolidation+ purpose_home_improvement \
        + purpose_house+ purpose_major_purchase+ purpose_medical \
       + purpose_moving+ purpose_other+ purpose_renewable_energy \
        + purpose_small_business+ purpose_vacation+ purpose_wedding' , data = finaldf)
results = model.fit() 
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               int_rate   R-squared:                       0.943
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                 1.071e+05
Date:                Mon, 02 Dec 2019   Prob (F-statistic):               0.00
Time:                        15:51:49   Log-Likelihood:            -2.7805e+05
No. Observations:              188181   AIC:                         5.562e+05
Df Residuals:                  188151   BIC:                         5.565e+05
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

In [26]:
#  checking out model to predict the test accuracy 
model = linear_model.LinearRegression()
fitted_model = model.fit(X = x_train_scaled[['loan_amnt'  , 'dti'  , 'inq_last_6mths' , 'pub_rec' , 'bc_util' , 'num_bc_tl' , 'num_tl_op_past_12m' , 'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E' , 'home_ownership_MORTGAGE' \
                , 'home_ownership_OTHER' , 'home_ownership_OWN' , \
       'home_ownership_RENT' , 'loan_status_Charged Off', 'purpose_credit_card', 'purpose_debt_consolidation' , 'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase', 'purpose_medical', 'purpose_moving', 'purpose_other', 'purpose_renewable_energy' \
        , 'purpose_small_business', 'purpose_vacation', 'purpose_wedding']], y = y_train)
print(fitted_model.coef_)
print(fitted_model.intercept_)

[ 1.05356560e-01  4.22660385e-02  1.07861939e-01  5.47869100e-02
  2.08162828e-01 -3.80891387e-02  8.96009682e-02 -5.62634011e+00
 -5.48685995e+00 -3.63101119e+00 -1.83331856e+00 -5.75191546e-01
 -1.47111104e-01 -8.26574455e-04 -4.47238389e-02 -6.65811135e-02
  4.24941998e-02 -1.64679719e-02  1.56805060e-02  8.18803627e-03
  1.34096469e-03  5.27648422e-03  2.17993093e-02  2.10691161e-02
  6.77627861e-02  6.12313304e-03  1.97632330e-02  2.04875301e-02
  1.60364781e-02]
14.24694547123541


In [27]:
Score_R2_train = fitted_model.score(X = x_train_scaled[['loan_amnt'  , 'dti'  , 'inq_last_6mths' , 'pub_rec' , 'bc_util' , 'num_bc_tl' , 'num_tl_op_past_12m' , 'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E' , 'home_ownership_MORTGAGE' \
                , 'home_ownership_OTHER' , 'home_ownership_OWN' , \
       'home_ownership_RENT' , 'loan_status_Charged Off', 'purpose_credit_card', 'purpose_debt_consolidation' , 'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase', 'purpose_medical', 'purpose_moving', 'purpose_other', 'purpose_renewable_energy' \
        , 'purpose_small_business', 'purpose_vacation', 'purpose_wedding']], y = y_train)
print('Training Accuracy:  ', Score_R2_train)
Score_R2 = fitted_model.score(X = x_test_scaled[['loan_amnt'  , 'dti'  , 'inq_last_6mths' , 'pub_rec' , 'bc_util' , 'num_bc_tl' , 'num_tl_op_past_12m' , 'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E' , 'home_ownership_MORTGAGE' \
                , 'home_ownership_OTHER' , 'home_ownership_OWN' , \
       'home_ownership_RENT' , 'loan_status_Charged Off', 'purpose_credit_card', 'purpose_debt_consolidation' , 'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase', 'purpose_medical', 'purpose_moving', 'purpose_other', 'purpose_renewable_energy' \
        , 'purpose_small_business', 'purpose_vacation', 'purpose_wedding']], y = y_test)
print('Testing Accuracy:  ',Score_R2)

Training Accuracy:   0.9427503365430738
Testing Accuracy:   0.9432286347084489
